Import all necessary libraries

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import pandas as pd
import numpy as np
from numpy import array

Import data 

In [20]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Repo/demand_final.csv')

In [21]:
df

,Time,Day ahead forecast,Hour ahead forecast,Current demand
0,00:00,21112.0,20527.0,20564.0
1,00:05,20262.0,20395.0,20510.0
2,00:10,20262.0,20395.0,20447.0
3,00:15,20262.0,20395.0,20335.0
4,00:20,20262.0,20160.0,20206.0
...,...,...,...,...
316739,23:40,22096.0,21565.0,21537.0
316740,23:45,22096.0,21565.0,21503.0
316741,23:50,22096.0,21258.0,21364.0
316742,23:55,22096.0,21258.0,21295.0


The LSTM model expects the input_shape to be three-dimensional matrix with [samples, timesteps, features], therefore, we must reshape the single input 
sample before making the prediction

Convert 'data' Dataframe to a numpy array

In [22]:
x_input = array(df['Current demand'])
x_input

array([20564., 20510., 20447., ..., 21364., 21295.,    nan])

Reshape 'data' array in order to be a 3-dimentional array. 
The array's prenious shape was (316744, 3) -> 316744 = 1096*289 features with a five minute timestep each for every day of a 3-year period. The new shape is (3, 289, 1096) -> 1096 days (3 years), 288 5-minute steps for each day +1, as 00:00 seems to be appeared twice, by 3 column of data (Hour ahead, Day ahead, Current).

In [23]:
n_steps = 289
n_features = 1096

In [24]:
x_input = x_input.reshape((n_features,n_steps, 1))
x_input

array([[[20564.],
        [20510.],
        [20447.],
        ...,
        [20680.],
        [20498.],
        [   nan]],

       [[20400.],
        [20315.],
        [20200.],
        ...,
        [21868.],
        [21761.],
        [   nan]],

       [[21308.],
        [21236.],
        [21230.],
        ...,
        [21570.],
        [21412.],
        [   nan]],

       ...,

       [[21147.],
        [21199.],
        [21226.],
        ...,
        [21222.],
        [21111.],
        [   nan]],

       [[21106.],
        [21024.],
        [21018.],
        ...,
        [20762.],
        [20679.],
        [   nan]],

       [[21125.],
        [21086.],
        [21070.],
        ...,
        [21364.],
        [21295.],
        [   nan]]])

In [25]:
outData = df.drop(['Time','Current demand', 'Day ahead forecast'], axis=1)
outData

,Hour ahead forecast
0,20527.0
1,20395.0
2,20395.0
3,20395.0
4,20160.0
...,...
316739,21565.0
316740,21565.0
316741,21258.0
316742,21258.0


In [26]:
y_output = array(outData)
y_output.shape

(316744, 1)

In [27]:
y_output = y_output.reshape((n_features,n_steps,1))
y_output

array([[[20527.],
        [20395.],
        [20395.],
        ...,
        [20645.],
        [20645.],
        [20645.]],

       [[20532.],
        [20300.],
        [20300.],
        ...,
        [21842.],
        [21842.],
        [21842.]],

       [[21387.],
        [21258.],
        [21258.],
        ...,
        [21473.],
        [21473.],
        [21473.]],

       ...,

       [[21177.],
        [21024.],
        [21024.],
        ...,
        [21116.],
        [21116.],
        [21116.]],

       [[21110.],
        [20971.],
        [20971.],
        ...,
        [20626.],
        [20626.],
        [20626.]],

       [[21237.],
        [21044.],
        [21044.],
        ...,
        [21258.],
        [21258.],
        [21258.]]])

Model construction


In [28]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, RepeatVector

In [32]:
model = Sequential()

# encoder layer
model.add(LSTM(289, activation='relu', input_shape=(n_steps, 1)))

# repeat Vector
model.add(RepeatVector(n_steps))

# decoder layer
model.add(LSTM(289, activation='relu', return_sequences=True))

model.add((Dense(1)))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 289)               336396    
                                                                 
 repeat_vector_3 (RepeatVect  (None, 289, 289)         0         
 or)                                                             
                                                                 
 lstm_7 (LSTM)               (None, 289, 289)          669324    
                                                                 
 dense_3 (Dense)             (None, 289, 1)            290       
                                                                 
Total params: 1,006,010
Trainable params: 1,006,010
Non-trainable params: 0
_________________________________________________________________


In [30]:
import tensorflow as tf
from keras.backend import binary_crossentropy
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), metrics=['accuracy'])

In [ ]:
history = model.fit(x_input, y_output, epochs=100, validation_split=0.1, verbose=1, batch_size=289)